# 台灣證券交易所爬蟲

## 以下用到的function:

pandas.to_numeric(series)  
apply 
lambda x: y   
pandas.set_index('~')  
df[]   

In [1]:
import requests 
import pandas as pd
from io import StringIO

response = requests.get('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201225&type=ALLBUT0999&_=1609052342267')

lines = response.text.split('\n')

lines[100]
newlines = []
for line in lines:
    if len(line.split('",')) == 17:
        newlines.append(line)
newlines[0:10]

df = pd.read_csv(StringIO("\n".join(newlines).replace("=","")))
df = df.astype(str)
df = df.apply(lambda s:s.str.replace(",",''))
df = df.set_index('證券代號')
#讓dataframe中的值變成數值
#errors='coerce'是指當data轉換為非數值時不會報錯
df = df.apply(lambda s:pd.to_numeric(s,errors='coerce'))

#如果我們在 sum 方法中傳遞了 axis=0，它將給出每列中出現 NaN 的次數。如果需要在每行中出現 NaN 次，我們需要設定 axis=1。
df = df[df.columns[df.isnull().sum() != len(df)]]
df.head()

df[df['收盤價']/df['開盤價']>1.05]

df.to_csv('daily_price.csv',encoding='utf_8_sig')
df = pd.read_csv('daily_price.csv',index_col=['證券代號'])
df = df.loc[~df['收盤價'].isnull()]
df

import sqlite3
conn = sqlite3.connect('test.sqlite3')
df.to_sql('daily_price',conn,if_exists='replace')
df = pd.read_sql('select * from daily_price',conn,index_col=['證券代號'])
df.head()

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
0050,2878357,3097,343128324,119.10,119.60,118.60,118.95,0.15,118.95,34,119.00,36,0.0
0051,78985,30,3395533,43.06,43.07,42.81,42.83,0.06,42.83,37,43.00,2,0.0
0052,1216995,524,129652996,107.00,107.30,106.05,106.50,0.35,106.50,9,106.65,1,0.0
0053,17287,14,970672,56.35,56.50,55.85,55.85,0.20,55.85,76,56.20,1,0.0
0054,16000,6,441810,27.75,27.75,27.57,27.57,0.02,27.57,58,27.75,2,0.0
